<a href="https://colab.research.google.com/github/Theophilus12/Training-An-Agent-to-Drive-Through-A-Roundabout-Without-Collision-Using-Reinforcement-Learning/blob/main/CarAgents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install stable-baselines

## importing the necessary software libraries 

In [ ]:
import gym
import os
import numpy as np
import highway_env
from matplotlib import pyplot as plt
%matplotlib inline
#from stable_baselines import HER, SAC, PP02, DQN
from stable_baselines3 import HerReplayBuffer, SAC, DDPG, TD3, PPO
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.evaluation import evaluate_policy 
from stable_baselines3.common.vec_env import DummyVecEnv

## Initializing & Testing the environment 

In [ ]:
env_name = "roundabout-v0"
env = gym.make(env_name)

model = PPO("MlpPolicy", env, verbose=1)
#model = PPO("MultiInputPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
# testing the environment 
episodes = 10
for episode in range(1, episodes+1):
    done = False
    score = 0
    #reset() function reset the environment to its initial stateThis approach restores the environment to its original condition and allows the agent to observe it.
    state = env.reset()
while not done:
#we can visually inspect the environmnet anytime by calling the render() function 
        env.render()
        action = env.action_space.sample()
        next_state, reward, done, info = env.step(action)
        score+=reward
print('Episode:{} Score:{}'.format(episode, score))
#print(info) 
#print(done)
#It completes the process and shuts down the environment, clearing up the resources inextricably linked with it.              
env.close()  

Episode:10 Score:9.68


lets evaluate the untrained agent

In [ ]:
#lets evaluate the untrained agent
eval_env = gym.make("roundabout-v0")
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes= 10)
print(f"mean_reward:{mean_reward:.2f}) +/- {std_reward:.2f}")

C:\Anaconda\envs\agent\lib\site-packages\stable_baselines3\common\evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward:9.06) +/- 0.10


## Understanding the environment 

There are two major spaces in an environment, ACTION SPACE(actions that the agent can take within the environment) and OBSERVATION SPACE(is it continuous or discrete, the range of values of each element in the observation, the python datatype of each element in the observation)

In [ ]:
env.action_space #we have 5 types of action that our agent can perform

Discrete(5)

In [ ]:
env.action_space.sample() 
# generating random actions, that our agent can perform

2

understaing the observation space nature
-> Box: gym data type describing numpy arrays with elements taking values continuously  in a range, 
(5,5)-> means that it is 5 by 5 dimensional vector NumPy array
range of element-> lower range[-inf] and upper range [+ inf] means the elements cantake values between -infinity and + infinity
datatype -> np.float32 

In [ ]:
env.observation_space

Box([[-inf -inf -inf -inf -inf]
 [-inf -inf -inf -inf -inf]
 [-inf -inf -inf -inf -inf]
 [-inf -inf -inf -inf -inf]
 [-inf -inf -inf -inf -inf]], [[inf inf inf inf inf]
 [inf inf inf inf inf]
 [inf inf inf inf inf]
 [inf inf inf inf inf]
 [inf inf inf inf inf]], (5, 5), float32)

In [ ]:
env.observation_space.sample()

array([[ 1.8947269 ,  0.26578045,  0.25293696, -0.24473181,  0.15543427],
       [ 0.12222444, -0.65149516, -0.68574   ,  0.2085767 ,  0.2860846 ],
       [-0.20846123, -0.63872725,  0.8249831 ,  0.3121078 , -0.06785931],
       [-0.1622476 ,  0.73136103, -2.407764  ,  2.1723502 ,  0.13067444],
       [ 0.24733485,  2.5595767 , -0.3607785 ,  0.00546531, -0.80610687]],
      dtype=float32)

In [ ]:
#This approach reset the environment to its initial state and start the simulation of the environment 
initial_state = env.reset()
initial_state

array([[ 1.0000000e+00,  2.0000000e-02,  4.4999999e-01,  0.0000000e+00,
        -5.3333336e-01],
       [ 1.0000000e+00, -5.4256640e-02,  1.9249992e-01,  9.4529361e-01,
         2.6643366e-01],
       [ 1.0000000e+00, -2.1317315e-01,  1.1025972e-01,  6.0933304e-01,
         1.0000000e+00],
       [ 1.0000000e+00,  1.0000000e+00, -2.0000000e-02, -9.3942398e-01,
         2.2204460e-16],
       [ 1.0000000e+00, -1.6246326e-01, -1.1664343e-01, -8.1259990e-01,
         1.0000000e+00]], dtype=float32)

NOTE: the render() function only works if reset() was called before, Otherwise it wont display the result

In [ ]:
import pprint

env = gym.make("highway-v0")
pprint.pprint(env.config)

{'action': {'type': 'DiscreteMetaAction'},
 'centering_position': [0.3, 0.5],
 'collision_reward': -1,
 'controlled_vehicles': 1,
 'duration': 40,
 'ego_spacing': 2,
 'high_speed_reward': 0.4,
 'initial_lane_id': None,
 'lane_change_reward': 0,
 'lanes_count': 4,
 'manual_control': False,
 'observation': {'type': 'Kinematics'},
 'offroad_terminal': False,
 'offscreen_rendering': False,
 'other_vehicles_type': 'highway_env.vehicle.behavior.IDMVehicle',
 'policy_frequency': 1,
 'real_time_rendering': False,
 'render_agent': True,
 'reward_speed_range': [20, 30],
 'right_lane_reward': 0.1,
 'scaling': 5.5,
 'screen_height': 150,
 'screen_width': 600,
 'show_trajectories': False,
 'simulation_frequency': 15,
 'vehicles_count': 50,
 'vehicles_density': 1}


In [ ]:
env = gym.make("roundabout-v0")
state = env.reset()
trajectory = []
for _ in range(1):
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    trajectory.append([state, action, reward, done, next_state])
    state = next_state
env.close()

print(f"Congrats! You just generated your first trajectory:\n{trajectory}")

Congrats! You just generated your first trajectory:
[[array([[ 1.0000000e+00,  2.0000000e-02,  4.4999999e-01,  0.0000000e+00,
        -5.3333336e-01],
       [ 1.0000000e+00, -2.6471149e-02,  1.9824046e-01,  9.6462458e-01,
         1.2880681e-01],
       [ 1.0000000e+00, -2.2279723e-01,  8.9226656e-02,  3.9587957e-01,
         9.8850363e-01],
       [ 1.0000000e+00,  1.0000000e+00, -2.0000000e-02, -8.8721341e-01,
         2.2204460e-16],
       [ 1.0000000e+00, -1.7194796e-01, -1.0214646e-01, -5.6082666e-01,
         9.4406599e-01]], dtype=float32), 2, 0.8800000000000001, False, array([[ 1.0000000e+00,  2.4060752e-02,  3.7019700e-01,  4.8730187e-02,
        -5.3110248e-01],
       [ 1.0000000e+00, -1.2613325e-01,  2.1319439e-01,  8.3840197e-01,
         6.5646183e-01],
       [ 1.0000000e+00,  1.1435085e-01,  1.7231613e-01,  8.8105738e-01,
        -4.1331553e-01],
       [ 1.0000000e+00, -2.0239449e-01,  5.5715024e-02,  1.2514360e-01,
         1.0000000e+00],
       [ 1.0000000e+00,  1

## Training the RL model

In [ ]:
#creating our Log Directory
log_path =os.path.join('Training', 'Logs')

In [ ]:
log_path

In [ ]:
#creating a model
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=20000)


## Save and Reload Model

In [3]:
#PP0_path = os.path.join('Traning', 'Saved Models', 'PPO_model')

In [ ]:
os.path.join(os.path.sep, 'Traning', 'Saved Models', 'PPO_model')

In [ ]:
model.save('roundabout')

In [ ]:
del model

In [ ]:
model= PPO.load('roundabout')

In [ ]:
#log_dir = "/tmp/gym/"

In [ ]:
#os.makedirs(log_dir, exist_ok=True)

## Evaluation

In [ ]:
#mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, render=True)
evaluate_policy(model, env, n_eval_episodes=10, render=True)
#print(f"mean_reward:{mean_reward:.2f}) +/- {std_reward:.2f}")

In [ ]:
env.close()

## Testing the Model

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

In [ ]:
env.close()

## Using an Alternate Algorithm to test the agent

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1)

In [ ]:
model.learn(total_timesteps=20000)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()